In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd # to import csv and for data manipulation
import matplotlib.pyplot as plt # to plot graph
import seaborn as sns # for intractve graphs
import numpy as np # for linear algebra
import datetime # to dela with date and time
%matplotlib inline
from sklearn.preprocessing import StandardScaler # for preprocessing the data
from sklearn.ensemble import RandomForestClassifier # Random forest classifier
from sklearn.tree import DecisionTreeClassifier # for Decision Tree classifier
from sklearn.svm import SVC # for SVM classification
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split # to split the data
from sklearn.cross_validation import KFold # For cross vbalidation
from sklearn.model_selection import GridSearchCV # for tunnig hyper parameter it will use all combination of given parameters
from sklearn.model_selection import RandomizedSearchCV # same for tunning hyper parameter but will use random combinations of parameters
from sklearn.metrics import confusion_matrix,recall_score,precision_recall_curve,auc,roc_curve,roc_auc_score,classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv("../input/creditcard.csv")

In [ ]:
data.head()

In [ ]:
!nvidia-smi

In [ ]:
"""Checking the class distribution"""

data["Class"].value_counts()

In [ ]:
sns.countplot("Class",data=data)

In [ ]:
Fraud_transacation = data[data["Class"]==1]
Normal_transacation= data[data["Class"]==0]
plt.figure(figsize=(10,6))
plt.subplot(121)
Fraud_transacation.Amount.plot.hist(title="Fraud Transacation")
plt.subplot(122)
Normal_transacation[Normal_transacation["Amount"]<=2500].Amount.plot.hist(title="Normal Transaction")

I am thinking to segregate the data into different clusters and then analysing cluster properties -> fraud transactions cluster in a separate cluster.
The approach I am going to to experiment with are:
1. SMOTE data genration
2. Droping least important features and genetrate new hand coded features (creating 2D 3D 4D fatures using the most important features)

New dimensions can also throw data in separate space and then Boosting models can be implemented very easily.

In [ ]:
from imblearn.over_sampling import SMOTE



In [ ]:
"""Converting data frame to numpy array to apply SMOTE"""
features= data.drop(["Class"],axis=1)
x=features.values
label = data["Class"]
y=label.values


In [ ]:
y.shape

In [ ]:
np.count_nonzero(y)

In [ ]:
sm = SMOTE(random_state=42)
X_res, Y_res = sm.fit_resample(x,y)

In [ ]:
print(np.count_nonzero(Y_res), "fraud count")

In [ ]:
"""Converting numpy array back to pandas data frame"""
X_s = pd.DataFrame(X_res)
Y_s = pd.DataFrame(Y_res)

In [ ]:
X_s.head()

In [ ]:
x_features_train,x_features_test,x_labels_train,x_labels_test =  train_test_split(X_s,Y_s,test_size=0.1)

In [ ]:
import xgboost as xgb
xgb_SMOTE = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(x_features_train,x_labels_train)

In [ ]:
predictions = xgb_SMOTE.predict(x_features_test)


In [ ]:
predictions = predictions.reshape(56863,1)
predictions.shape

In [ ]:
x_labels_test.shape

In [ ]:
"""The accuracy achived is 99.182"""
from sklearn.metrics import accuracy_score
accuracy_score(x_labels_test, predictions)

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        1#print('Confusion matrix, without normalization')

    #print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cnf_matrix = confusion_matrix(x_labels_test, predictions)
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()

In [ ]:
xgb_SMOTE.feature_importances_

In [ ]:
X_s.head()

In [ ]:
X_s[30] = pd.DataFrame(X_res[:,4]*X_res[:,14])
X_s[31] = pd.DataFrame(X_res[:,0]*X_res[:,12])
X_s[32] = pd.DataFrame(X_res[:,4]*X_res[:,14]*X_res[:,12])
X_s[33] = pd.DataFrame(X_res[:,0]*X_res[:,12]*X_res[:,4])
X_s[34] = pd.DataFrame(X_res[:,0]*X_res[:,17])
X_s[35] = pd.DataFrame(X_res[:,4]*X_res[:,17]*X_res[:,14])

In [ ]:
X_s.head()

In [ ]:
x_features_train1,x_features_test1,x_labels_train1,x_labels_test1 =  train_test_split(X_s,Y_s,test_size=0.1)

In [ ]:
import xgboost as xgb
xgb_feature = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(x_features_train1,x_labels_train1)

In [ ]:
xgb_feature = xgb.XGBClassifier(max_depth=5, n_estimators=350, learning_rate=0.02).fit(x_features_train1,x_labels_train1)

In [ ]:
predictions1 = xgb_feature.predict(x_features_test1)
predictions1 = predictions1.reshape(56863,1)

In [ ]:
cnf_matrix = confusion_matrix(x_labels_test1, predictions1)
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()

In [ ]:
features =list(X_s.columns.values)
features_str = [str(item) for item in features]
X_s.columns=[features_str]
X_s['LABEL']=Y_s
X_s.head()

In [ ]:
import tensorflow as tf

In [ ]:
def make_train_input_fn(dfx, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [ ]:

def make_eval_input_fn(dfx,dfy):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

In [ ]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

In [ ]:
COLUMNS= ['1','2','3','4','5']

In [ ]:
FEATURES = COLUMNS[0:len(COLUMNS)]
print(FEATURES)

In [ ]:
def make_feature_cols():
  input_columns = [tf.feature_column.re(k) for k in FEATURES]
  return input_columns

In [ ]:
import shutil

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(X_s.head(),Y_s.head(), num_epochs = 10))